# JavaScript Inclusion Issues experiment Demo Analysis

## General Analysis

In [ ]:
# imports
from database import Task, URL
import matplotlib.pyplot as plt
import pandas
from IPython.display import display, HTML

In [ ]:
# some filtering
task = Task.select().where(Task.job == "demoinclusions")
url = URL.select().join(Task).where(URL.task.job == "demoinclusions")

In [ ]:
# general statistics
general_stats = pandas.DataFrame.from_dict({
    "": ["with session", "without session", "total"],
    "crawled sites":  [task.where(Task.session != None).count(), task.where(Task.session == None).count(), task.count()],
    "crawled urls": [url.where(Task.session != None).count(), url.where(Task.session == None).count(), url.count()],
})
display(HTML(general_stats.to_html(index=False)))

## Inclusion Issues Analysis

In [ ]:
# imports
from database import Task, URL
from modules.inclusionissues import Script, ScriptInclusion
import matplotlib.pyplot as plt
import pandas
from IPython.display import display, HTML
from peewee import fn

In [ ]:
# some filtering
task = Task.select().where(Task.job == "demoinclusions")
url = URL.select().join(Task).where(URL.task.job == "demoinclusions")
script = Script.select()
def inclusion(*x):
    return ScriptInclusion.select(*x).join(Script).switch(ScriptInclusion).join(Task)

In [ ]:
# general statistics for the experiment
general_stats = pandas.DataFrame.from_dict({
    "": ["with session", "without session", "total"],
    "unique scripts":  [
        inclusion(tuple([fn.Count(Script.id)])).group_by([Script.id]).where(Task.session != None).count(),
        inclusion(tuple([fn.Count(Script.id)])).group_by([Script.id]).where(Task.session == None).count(),
        inclusion(tuple([fn.Count(Script.id)])).group_by([Script.id]).count()
    ],
    "script inclusions": [
        inclusion().where(Task.session != None).count(),
        inclusion().where(Task.session == None).count(),
        inclusion().count()
    ],
    "dynamic/inclusion": [
        inclusion().where(ScriptInclusion.inclusion_type == "dynamic/inclusion").where(Task.session != None).count(),
        inclusion().where(ScriptInclusion.inclusion_type == "dynamic/inclusion").where(Task.session == None).count(),
        inclusion().where(ScriptInclusion.inclusion_type == "dynamic/inclusion").count()
    ],
    "eval": [
        inclusion().where(ScriptInclusion.inclusion_type == "eval").where(Task.session != None).count(),
        inclusion().where(ScriptInclusion.inclusion_type == "eval").where(Task.session == None).count(),
        inclusion().where(ScriptInclusion.inclusion_type == "eval").count()
    ],
    "inclusion": [
        inclusion().where(ScriptInclusion.inclusion_type == "inclusion").where(Task.session != None).count(),
        inclusion().where(ScriptInclusion.inclusion_type == "inclusion").where(Task.session == None).count(),
        inclusion().where(ScriptInclusion.inclusion_type == "inclusion").count()
    ],
    "inline-script": [
        inclusion().where(ScriptInclusion.inclusion_type == "inline-script").where(Task.session != None).count(),
        inclusion().where(ScriptInclusion.inclusion_type == "inline-script").where(Task.session == None).count(),
        inclusion().where(ScriptInclusion.inclusion_type == "inline-script").count()
    ]
})
display(HTML(general_stats.to_html(index=False)))